In [1]:
#!pip install imgaug==0.2.6 -q
#!pip install PyYAML==5.3 -q
#!pip install torchmetrics==0.2.0 -q
#!pip install pytorch-lightning
#!pip install lightning-flash -U

In [2]:
n_steps = 3
n_features = 1

In [3]:
import pandas as pd

msft_df = pd.read_csv('/content/drive/MyDrive/MSFT.csv')
msft_df = msft_df.T
msft_df.drop(['Unnamed: 0'], inplace=True)
msft_df.drop(columns=[0,1,2,4], inplace= True)
msft_df.rename(columns={3: 'y'}, inplace='True')
msft_df=msft_df.iloc[::-1]
msft_df

,y
1999-11-12,89.19
1999-11-19,86
1999-11-26,91.12
1999-12-03,96.12
1999-12-10,93.87
...,...
2021-03-26,236.48
2021-04-01,242.35
2021-04-09,255.85
2021-04-16,260.74


In [4]:
msft_df.y = msft_df.y.astype('float32')
msft_df.y.dtype

dtype('float32')

In [5]:
from sklearn.model_selection import TimeSeriesSplit

X = msft_df.y.values[:-1]
y = msft_df.y.values[1:]

tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188] TEST: [189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242


In [6]:
X_train, X_test
y_train, y_test

(array([ 86.    ,  91.12  ,  96.12  ,  93.87  , 115.25  , 117.44  ,
        116.75  , 111.44  , 112.25  , 103.75  ,  98.25  , 106.56  ,
         99.94  ,  95.06  ,  91.31  ,  96.12  , 101.    ,  99.37  ,
        111.69  , 106.25  ,  89.06  ,  74.12  ,  78.94  ,  69.75  ,
         71.12  ,  68.81  ,  65.06  ,  61.44  ,  66.31  ,  68.81  ,
         72.56  ,  77.69  ,  80.    ,  82.    ,  78.94  ,  72.31  ,
         69.69  ,  69.12  ,  72.44  ,  71.    ,  70.62  ,  70.19  ,
         69.31  ,  64.19  ,  63.25  ,  60.31  ,  55.56  ,  53.75  ,
         65.19  ,  67.69  ,  68.25  ,  67.37  ,  69.06  ,  69.94  ,
         56.63  ,  54.44  ,  49.19  ,  46.44  ,  43.38  ,  49.13  ,
         53.5   ,  61.    ,  64.    ,  60.81  ,  59.13  ,  57.31  ,
         56.75  ,  56.69  ,  56.69  ,  54.56  ,  56.56  ,  54.69  ,
         56.19  ,  62.18  ,  69.    ,  67.12  ,  70.75  ,  69.4   ,
         68.09  ,  70.91  ,  70.34  ,  73.19  ,  68.02  ,  68.83  ,
         73.    ,  66.06  ,  71.34  ,  69.18  , 

In [7]:
import numpy as np
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x = timeseries_data[i:end_ix]
		X.append(seq_x)
	return np.array(X)

X_train = prepare_data(X_train, n_steps)
X_test = prepare_data(X_test, n_steps) 

In [8]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf

checkpoint_path = "/content/drive/MyDrive/TF_stock_forecaster.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)



model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mae')
# fit model

model.fit(X_train, y_train, epochs=300, batch_size=64, verbose=1, callbacks=[cp_callback], validation_split = 0.1)

Epoch 1/300
14/14 [==============================] - 3s 53ms/step - loss: 36.3039 - val_loss: 49.8850

Epoch 00001: val_loss improved from inf to 49.88502, saving model to /content/drive/MyDrive/TF_stock_forecaster.ckpt
Epoch 2/300
14/14 [==============================] - 0s 9ms/step - loss: 28.7069 - val_loss: 19.5304

Epoch 00002: val_loss improved from 49.88502 to 19.53037, saving model to /content/drive/MyDrive/TF_stock_forecaster.ckpt
Epoch 3/300
14/14 [==============================] - 0s 8ms/step - loss: 7.2480 - val_loss: 3.9383

Epoch 00003: val_loss improved from 19.53037 to 3.93829, saving model to /content/drive/MyDrive/TF_stock_forecaster.ckpt
Epoch 4/300
14/14 [==============================] - 0s 10ms/step - loss: 1.7955 - val_loss: 0.8789

Epoch 00004: val_loss improved from 3.93829 to 0.87888, saving model to /content/drive/MyDrive/TF_stock_forecaster.ckpt
Epoch 5/300
14/14 [==============================] - 0s 9ms/step - loss: 0.9552 - val_loss: 0.7684

Epoch 00005: v

In [14]:
model.evaluate(x=X_test, y=y_test[:-3], batch_size=8, verbose=1)

23/23 [==============================] - 0s 2ms/step - loss: 0.3407


0.340738981962204

In [19]:
pred = model.predict(X_test)
print(len(pred), len(X_train))

183 930
